In [1]:
#hide
#skip
%config Completer.use_jedi = False
# upgrade fastrl on colab
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

In [2]:
# hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON
else:
    # Virutual display is needed for colab
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(400, 300))
    display.start()

In [3]:
# default_exp memory.experience_replay

In [4]:
# export
# Python native modules
import os
# Third party libs
from fastcore.all import *
from fastai.learner import *
from fastai.torch_basics import *
from fastai.torch_core import *
from fastai.callback.all import *
# Local modules
from fastrl.core import *
from fastrl.callback.core import *
from fastrl.data.block import *

# Experience Replay
> Experience Replay is likely the simplest form of memory used by RL agents. 

In [5]:
# export
class ExperienceReplay(Callback):
    def __init__(self,*args,bs=16,max_sz=200,warmup_sz=100,**kwargs):
        store_attr()
        self.memory=None
        self.pointer=0
    
    def after_pred(self):
        xb=BD(self.learn.xb[0]).mapv(to_detach)
        if self.memory is None:            self.memory=xb
        elif self.memory.bs()<self.max_sz: self.memory+=xb
        else:
            self.memory=self.memory[:self.pointer]+xb+self.memory[self.pointer+xb.bs():]
            self.pointer+=xb.bs()
            if self.pointer>self.max_sz: self.pointer=0
        with torch.no_grad():
            idxs=np.random.randint(0,self.memory.bs(),self.bs).tolist()
            self.learn.xb=(self.memory[idxs].mapv(to_device),)
        
        if self.memory.bs()<self.warmup_sz: raise CancelBatchException

In [6]:
from fastrl.data.gym import *
source=Source(
    cbs=[GymLoop(env_name='CartPole-v1',steps_delta=1,steps_count=1,seed=0),FirstLast]
)

In [7]:
source=Source(cbs=[GymLoop(env_name='CartPole-v1',steps_delta=1,steps_count=1,seed=0),FirstLast])
learn=fake_gym_learner(source,n=20,bs=1)

Could not do one pass in your dataloader, there is something wrong in it


In [20]:
experience_replay=ExperienceReplay()
experience_replay.learn=learn

In [21]:
for b in learn.dls[0]:
    learn.xb=b
    
    experience_replay.after_pred()

CancelBatchException: 

In [9]:
# hide
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.export import *
    from nbdev.export2html import *
    from nbdev.cli import make_readme
    make_readme()
    notebook2script()
    notebook2html()

converting /home/fastrl_user/fastrl/nbs/index.ipynb to README.md
Converted 00_core.ipynb.
Converted 00_nbdev_extension.ipynb.
Converted 03_callback.core.ipynb.
Converted 04_agent.ipynb.
Converted 05_data.test_async.ipynb.
Converted 05a_data.block.ipynb.
Converted 05b_data.gym.ipynb.
Converted 06a_memory.experience_replay.ipynb.
Converted 10a_agents.dqn.core.ipynb.
Converted 10b_agents.dqn.targets.ipynb.
Converted 10c_agents.dqn.double.ipynb.
Converted 10d_agents.dqn.dueling.ipynb.
Converted 10e_agents.dqn.categorical.ipynb.
Converted 11a_agents.policy_gradient.ppo.ipynb.
Converted 20_test_utils.ipynb.
Converted index.ipynb.
Converted nbdev_template.ipynb.
converting: /home/fastrl_user/fastrl/nbs/06a_memory.experience_replay.ipynb
